# Childrens Act Disposals

In [3]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re
import os  # for file paths
import awswrangler as wr

In [4]:
# Structured  dataframes
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [5]:
# Assigning databases and S3 bucket
db1 = "familyman_dev_v2"
fcsq_db = "fcsq"
# create path for within athena FCSQ database in the S3 folder
fcsq_db_path = f"s3://alpha-family-data/fcsq_processing/CA_disps/"
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")
snapshot_date = "2021-11-11"

### Orders made

In [14]:
pydb.create_temp_table( 
f"""
  SELECT 
    DISTINCT
    case_number,
    disp_date,
    Year,
    Quarter,
    case_type,
    receipt_date,
    entry_date,
    event,
    event_model,
    field_model,
    order_type,
    disp_type_code,
    disp_type,
    order_code,
    order_desc,
    creating_court
  FROM 
    {fcsq_db}.ca_disps a 
  WHERE 
    disp_type_code = 1
""",

"ca_disp_order_type")

In [13]:
pydb.read_sql_query ("select count (*) count from __temp__.ca_order_type where year = 2020 and case_type = 'P' and order_code = 29 ")

,count
0,46368


### Disposal Events

In [24]:
pydb.create_temp_table( 
f"""
  SELECT 
    DISTINCT
    case_number,
    disp_date,
    Year,
    Quarter,
    case_type,
    receipt_date,
    entry_date,
    event,
    event_model,
    field_model,
    order_type,
    disp_type_code,
    disp_type,
    order_code,
    order_desc,
    creating_court,
    FO_value
  FROM 
    {fcsq_db}.ca_disps a 
  WHERE 
    order_code < 40
""",

"ca_disp_events")

In [21]:
pydb.read_sql_query ("select count (*) count from __temp__.ca_disp_events where year = 2011 and case_type = 'P'")

,count
0,122138


In [25]:
pydb.read_sql_query ("select *  from __temp__.ca_disp_events limit 10")

,case_number,disp_date,year,quarter,case_type,receipt_date,entry_date,event,event_model,field_model,order_type,disp_type_code,disp_type,order_code,order_desc,creating_court,fo_value
0,IP10P00643,2011-01-12,2011,1,P,2011-01-12,2011-01-12 15:45:40,23300593578,C43,C43_5,RS,1,Order,30,Section 8 Residence Order,IP,Y
1,HM05P00259,2005-06-22,2005,2,P,2005-06-22,2005-06-23 13:04:21,22700208340,C43,C43_5,RS,1,Order,30,Section 8 Residence Order,HM,N
2,RG01P02176,2003-05-12,2003,2,P,2003-05-12,2003-05-13 14:34:57,30500263847,C43,C43_5,CN,1,Order,29,Section 8 Contact Order,RG,N
3,MB05P00822,2005-11-24,2005,4,P,2005-11-24,2005-12-05 11:09:24,27000460548,C43,C43_5,RS,1,Order,30,Section 8 Residence Order,MB,Y
4,WV20P01681,2021-01-29,2021,1,P,2021-01-29,2021-02-03 11:33:13,37802375239,C43,C43_5,PS,1,Order,31,Section 8 Prohibited Steps Order,TF,N
5,WR20P00174,2021-01-21,2021,1,P,2021-01-21,2021-02-01 14:34:14,38001699207,C43,C43_5,CAST,1,Order,29,Section 8 Contact Order,WR,Y
6,PO05P00312,2005-09-08,2005,3,P,2005-09-08,2005-09-14 10:36:02,30202098580,C43,C43_5,CN,1,Order,29,Section 8 Contact Order,PO,Y
7,AD10C10005,2011-11-21,2011,4,C,2011-11-21,2011-11-23 12:51:51,56500007072,C32A,<NA>,N/A,1,Order,1,Care Order/Substitute Supervision Order,LI,Y
8,UW11P00072,2012-04-26,2012,2,P,2012-04-26,2012-05-29 14:32:54,55600024412,C43,C43_5,CN,1,Order,29,Section 8 Contact Order,SN,N
9,MA17P50021,2017-08-02,2017,3,P,2017-08-02,2017-08-11 16:24:13,33600698901,C42,<NA>,N/A,1,Order,33,Family Assistance Order,SK,Y


In [34]:
pydb.create_temp_table(
f"""
Select
    *,
    ROW_NUMBER() OVER(PARTITION BY case_number, fo_value
                       ORDER BY case_number, disp_date ASC) 
      AS min_date,
    ROW_NUMBER() OVER(PARTITION BY case_number, fo_value
                       ORDER BY case_number, disp_date DESC) 
      AS max_date  
FROM
    __temp__.ca_disp_events
""",

"ca_FO_dates")
 
    

In [35]:
pydb.read_sql_query ("select * from __temp__.ca_FO_dates limit 30")

,case_number,disp_date,year,quarter,case_type,receipt_date,entry_date,event,event_model,field_model,order_type,disp_type_code,disp_type,order_code,order_desc,creating_court,fo_value,min_date,max_date
0,CV01P00209,2001-06-14,2001,2,P,2001-06-14,2001-06-15 07:59:25,18000203753,C43,C43_5,RS,1,Order,30,Section 8 Residence Order,CV,N,2,2
1,CV01P00445,2001-09-18,2001,3,P,2001-09-18,2001-09-18 12:16:17,18000224380,C43,C43_5,RS,1,Order,30,Section 8 Residence Order,CV,N,1,1
2,CV02C00111,2003-02-11,2003,1,C,2003-02-11,2003-02-11 15:35:27,18000361333,C32A,<NA>,N/A,1,Order,1,Care Order/Substitute Supervision Order,CV,N,1,1
3,CV02P00234,2002-05-01,2002,2,P,2002-05-01,2002-05-02 14:24:45,18000278480,C43,C43_5,RS,1,Order,30,Section 8 Residence Order,CV,N,3,1
4,CV02P00234,2002-05-01,2002,2,P,2002-05-01,2002-05-02 14:23:53,18000278466,C45A,<NA>,N/A,1,Order,25,Parental Responsibility Order,CV,N,4,2
5,CV02P00234,2002-04-11,2002,2,P,2002-04-11,2002-04-12 11:53:52,18000272766,C45A,<NA>,N/A,1,Order,25,Parental Responsibility Order,CV,N,1,3
6,CV02P00234,2002-04-11,2002,2,P,2002-04-11,2002-04-12 11:48:02,18000272764,C43,C43_5,RS,1,Order,30,Section 8 Residence Order,CV,N,2,4
7,CV02P00687,2003-09-18,2003,3,P,2003-09-18,2003-09-19 08:40:42,18000427216,C30,<NA>,N/A,1,Order,20,Missing Child Info,CV,N,4,1
8,CV02P00687,2002-08-08,2002,3,P,2002-08-08,2002-08-09 17:22:06,18000306969,C43,C43_5,RS,1,Order,30,Section 8 Residence Order,CV,N,1,2
9,CV02P00687,2002-08-08,2002,3,P,2002-08-08,2002-08-09 17:30:39,18000306970,C45A,<NA>,N/A,1,Order,25,Parental Responsibility Order,CV,N,2,3
